# SIADS 591-592 Milestone I-II Dog Breed Popularity Analysis

This notebook will contain the analysis of dog breed popularity data

## Python Library Loading

In [1]:
# Install packages that are needed that are currently not part of the environment
#! pip install lxml
#! pip install beautifulsoup4
! pip install altair
! pip install vega_datasets

In [2]:
# Import required python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lxml
from bs4 import BeautifulSoup
import altair as alt
from io import StringIO
from vega_datasets import data

## Data Loading

### Web Scraping

In [3]:
# When trying to connect to the web page for web scraping, there is a proxy error so the page was downloaded locally for loading
with open('project_data/MostPopularDogBreeds_ DogBreedPopularity2018.html','r') as akc_html:
    content = akc_html.read()
    soup = BeautifulSoup(content,'lxml')
    table = soup.find('table', class_='content-body__responsive-table')#.replace(' ','')
    title = ''
    for row in table.find_all('th'):
        title = title + row.text + ','
    
    body =''
    for row in table.find_all('tr'):
        for cell in row.find_all('td'):
            body = body + cell.text +','
        body = body + '\n'
        
    akc = title + body

In [4]:
popularity_df = pd.read_csv(StringIO(akc), sep=",")
popularity_df.drop(columns = 'Unnamed: 6', inplace=True)
popularity_df.head()

,Breed,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank
0,Labrador Retrievers,1,1.0,1.0,1.0,1.0
1,German Shepherd Dogs,2,2.0,2.0,2.0,2.0
2,Golden Retrievers,3,3.0,3.0,3.0,3.0
3,French Bulldogs,4,4.0,6.0,6.0,9.0
4,Bulldogs,5,5.0,4.0,4.0,4.0


### Loading CSV Files Using Pandas

In [5]:
# Read in the data sets
best_show_df = pd.read_csv("project_data/best_in_show.csv", skiprows=[1])
best_show_df.drop(list(best_show_df.filter(regex = 'Unnamed')), axis = 1, inplace = True)
best_show_df.rename(columns={"Dog breed": "Breed"}, inplace=True)
best_show_df.head()

,Breed,category,datadog score,POPULARITY IN US,POPULARITY IN US.1,"LIFETIME COST, $",5 LIFETIME COST,1 INTELLIGENCE (TRAINABILITY) ranking,INTELLIGENCE (TRAINABILITY) ranking,2 LONGEVITY,...,"food per lifetime, $","Other regular costs, total per lifetime, $","total per year, $","total, per year, £","toys, presents, treats, per year, £","pet sitters, per year, £","grooming, per year, £","vet fees per year, £","kennels per year, £","one offs, $"
0,Border Collie,herding,3.64,45.0,39.0,"$20,143",48%,1,100%,12.52,...,"3,486","13,095","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
1,Border Terrier,terrier,3.61,80.0,61.0,"$22,638",14%,30,70%,14.00,...,"3,898","14,643","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
2,Brittany,sporting,3.54,30.0,30.0,"$22,589",16%,19,80%,12.92,...,"5,171","13,514","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
3,Cairn Terrier,terrier,3.53,59.0,48.0,"$21,992",22%,35,61%,13.84,...,"3,854","14,476","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
4,Welsh Springer Spaniel,sporting,3.34,130.0,81.0,"$20,224",47%,31,69%,12.49,...,"3,478","13,064","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0


In [6]:
breed_info_df = pd.read_csv("project_data/AKC_Breed_Info.csv", encoding= 'unicode_escape')
breed_info_df.head()

,Breed,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs
0,Akita,26,28,80,120
1,Anatolian Sheepdog,27,29,100,150
2,Bernese Mountain Dog,23,27,85,110
3,Bloodhound,24,26,80,120
4,Borzoi,26,28,70,100


In [7]:
#add two more datasets
# https://www.rd.com/list/most-popular-dog-breeds-in-every-state/
top_dog_by_state = pd.read_csv("project_data/top_dog_breeds_by_state.csv")
top_dog_by_state.head()

,State,Top 1,Top 2,Top 3
0,Alabama,Labrador retriever,German shepherd,Beagle
1,Alaska,Labrador retriever,German shepherd,Golden retriever
2,Arizona,Labrador retriever,German shepherd,Golden retriever
3,Arkansas,Labrador retriever,German shepherd,Beagle
4,California,Labrador retriever,French bulldog,German shepherd


In [8]:
# https://data.world/len/dog-size-intelligence-linked/workspace/file?filename=dog_intelligence.csv
dog_iq = pd.read_csv("project_data/dog_intelligence.csv")
dog_iq.head()

,Breed,Classification,obey,reps_lower,reps_upper
0,Border Collie,Brightest Dogs,95%,1,4
1,Poodle,Brightest Dogs,95%,1,4
2,German Shepherd,Brightest Dogs,95%,1,4
3,Golden Retriever,Brightest Dogs,95%,1,4
4,Doberman Pinscher,Brightest Dogs,95%,1,4


### Generate Some Mean Statistics

In [9]:
popularity_df["Mean Rank"] = popularity_df.apply(lambda row: np.mean(row[1:]), axis=1)
popularity_df["2014-2015 Change"] = -(popularity_df["2015 Rank"] - popularity_df["2014 Rank"])
popularity_df["2015-2016 Change"] = -(popularity_df["2016 Rank"] - popularity_df["2015 Rank"])
popularity_df["2016-2017 Change"] = -(popularity_df["2017 Rank"] - popularity_df["2016 Rank"])
popularity_df["2017-2018 Change"] = -(popularity_df["2018 Rank"] - popularity_df["2017 Rank"])
popularity_df.replace(-0.0, 0.0, inplace=True)
popularity_df.head()

,Breed,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,Mean Rank,2014-2015 Change,2015-2016 Change,2016-2017 Change,2017-2018 Change
0,Labrador Retrievers,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,German Shepherd Dogs,2,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
2,Golden Retrievers,3,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0
3,French Bulldogs,4,4.0,6.0,6.0,9.0,5.8,3.0,0.0,2.0,0.0
4,Bulldogs,5,5.0,4.0,4.0,4.0,4.4,0.0,0.0,-1.0,0.0


In [10]:
cols=[i for i in breed_info_df.columns if i not in ["Breed"]]
for col in cols:
    breed_info_df[col]=pd.to_numeric(breed_info_df[col], errors='coerce')
breed_info_df.dropna(inplace=True)
breed_info_df["Mean Height"] = breed_info_df.apply(lambda row: np.mean(row[1:3]), axis=1)
breed_info_df["Mean Weight"] = breed_info_df.apply(lambda row: np.mean(row[3:5]), axis=1)
breed_info_df.head()

,Breed,height_low_inches,height_high_inches,weight_low_lbs,weight_high_lbs,Mean Height,Mean Weight
0,Akita,26.0,28.0,80.0,120.0,27.0,100.0
1,Anatolian Sheepdog,27.0,29.0,100.0,150.0,28.0,125.0
2,Bernese Mountain Dog,23.0,27.0,85.0,110.0,25.0,97.5
3,Bloodhound,24.0,26.0,80.0,120.0,25.0,100.0
4,Borzoi,26.0,28.0,70.0,100.0,27.0,85.0


## Data Cleaning and Merging

In [11]:
# The popularity data frame has the dog breed names in plural for all breeds. The trailing 's' needs to be removed
popularity_df['Breed'] = popularity_df['Breed'].str.strip('s')
popularity_df['Breed'] = popularity_df['Breed'].str.strip(" ")
print(sorted(popularity_df['Breed']))

['Affenpinscher', 'Afghan Hound', 'Airedale Terrier', 'Akita', 'Alaskan Malamute', 'American English Coonhounds', 'American Eskimo Dog', 'American Foxhounds', 'American Hairless Terrier', 'American Staffordshire Terrier', 'American Water Spaniels', 'Anatolian Shepherd Dog', 'Australian Cattle Dog', 'Australian Shepherd', 'Australian Terrier', 'Basenji', 'Basset Hound', 'Beagle', 'Bearded Collie', 'Beauceron', 'Bedlington Terrier', 'Belgian Malinoi', 'Belgian Sheepdog', 'Belgian Tervuren', 'Bergamasco Sheepdogs', 'Berger Picard', 'Bernese Mountain Dog', 'Bichon Frise', 'Black Russian Terrier', 'Black and Tan Coonhounds', 'Bloodhound', 'Bluetick Coonhound', 'Boerboel', 'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier', 'Bouviers des Flandre', 'Boxer', 'Boykin Spaniel', 'Briard', 'Brittany', 'Brussels Griffon', 'Bull Terrier', 'Bulldog', 'Bullmastiff', 'Cairn Terrier', 'Canaan Dogs', 'Cani Corsi', 'Cardigan Welsh Corgi', 'Cavalier King Charles Spaniel', 'Cesky Terriers', 'Ches

In [12]:
popularity_df.head()

,Breed,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,Mean Rank,2014-2015 Change,2015-2016 Change,2016-2017 Change,2017-2018 Change
0,Labrador Retriever,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,German Shepherd Dog,2,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0
2,Golden Retriever,3,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0
3,French Bulldog,4,4.0,6.0,6.0,9.0,5.8,3.0,0.0,2.0,0.0
4,Bulldog,5,5.0,4.0,4.0,4.0,4.4,0.0,0.0,-1.0,0.0


In [13]:
# Lets see what dogs are unique in the popularity and breed info frames
popularity_unique = [x for x in set(popularity_df['Breed']) if x not in set(breed_info_df['Breed'])]
breed_info_unique = [x for x in set(breed_info_df['Breed']) if x not in set(popularity_df['Breed'])]
print("Popularity unique: {}".format(sorted(popularity_unique)))
print("Breed Info unique: {}".format(sorted(breed_info_unique)))

Popularity unique: ['Airedale Terrier', 'Alaskan Malamute', 'American English Coonhounds', 'American Eskimo Dog', 'American Foxhounds', 'American Hairless Terrier', 'American Water Spaniels', 'Anatolian Shepherd Dog', 'Belgian Malinoi', 'Bergamasco Sheepdogs', 'Berger Picard', 'Black and Tan Coonhounds', 'Bluetick Coonhound', 'Boerboel', 'Bouviers des Flandre', 'Boykin Spaniel', 'Bulldog', 'Canaan Dogs', 'Cani Corsi', 'Cesky Terriers', 'Chinese Shar-Pei', 'Chinook', 'Cirnechi dell’Etna', 'Cocker Spaniel', 'Collie', 'Coton de Tulear', 'Curly-Coated Retrievers', 'Dandie Dinmont Terriers', 'Dogues de Bordeaux', 'English Cocker Spaniel', 'English Foxhounds', 'Entlebucher Mountain Dog', 'Finnish Lapphund', 'Flat-Coated Retriever', 'Glen of Imaal Terriers', 'Grand Basset Griffon Vendeens', 'Great Pyrenee', 'Greater Swiss Mountain Dog', 'Havanese', 'Icelandic Sheepdogs', 'Irish Red and White Setter', 'Irish Water Spaniels', 'Keeshonden', 'Komondorok', 'Kuvaszok', 'Lagotti Romagnoli', 'Leonber

In [14]:
# We can see that there are some breed that are named similarly but just need to be renamed in each so they match
# For example, there are multiple spellings of Airedale Terrier
# Let's try and rescue some of this data by creating a dictionary where the key is the current entry and the value is what it 
# should be corrected to
correction_dict = {
    "Airdale Terrier": "Airedale Terrier",
    "American English Coonhounds": "American English Coonhound",
    "American Eskimo": "American Eskimo Dog",
    "American Foxhounds": "American Foxhound",
    "American Water Spaniels": "American Water Spaniel",
    "Anatolian Sheepdog": "Anatolian Shepherd Dog",
    "Belgian Malinois": "Belgian Malinoi",
    "Black and Tan Coonhounds": "Black and Tan Coonhound",
    "Black and Tan Coonhound": "Black And Tan Coonhound",
    "Bouvier Des Flandres": "Bouviers des Flandre",
    "Bull Dog": "Bulldog",
    "Canaan Dogs": "Canaan Dog",
    "Cane Corso": "Cani Corsi",
    "Cesky Terriers": "Cesky Terrier",
    "Chinese Shar-Pei": "Chinese Shar Pei",
    "Cocker Spaniel-American": "Cocker Spaniel",
    "Collie (Rough) & (Smooth)": "Collie",
    "Curly-Coated Retrievers": "Curly Coated Retriever",
    "Dandie Dinmont Terriers": "Dandie Dinmont Terrier",
    "English Foxhounds": "English Foxhound",
    "Flat Coated Retriever": "Flat-Coated Retriever",
    "German Shepherd": "German Shepherd Dog",
    "Glen of Imaal Terriers": "Glen of Imaal Terrier",
    "Great Pyrenees": "Great Pyrenee",
    "Great Swiss Mountain Dog": "Greater Swiss Mountain Dog",
    "Icelandic Sheepdogs": "Icelandic Sheepdog",
    "Irish Water Spaniels": "Irish Water Spaniel",
    "Keeshonden": "Keeshond",
    "Komondorok": "Komondor",
    "Kuvaszok": "Kuvasz",
    "Löwchen": "Lowchen",
    "Manchester Terrier (Standard)": "Manchester Terrier",
    "Miniature Schnauzers": "Miniature Schnauzer",
    "Neopolitan Mastiff": "Neapolitan Mastiff",
    "Norwegian Buhunds": "Norwegian Buhund",
    "Old English Sheepdog (Bobtail)": "Old English Sheepdog",
    "Otter Hound": "Otterhound",
    "Pharaoh Hounds": "Pharaoh Hound",
    "Plott Hounds": "Plott Hound",
    "Plott": "Plott Hound",
    "Polish Lowland Sheepdogs": "Polish Lowland Sheepdog",
    "Poodle Standard": "Poodle",
    "Pulik": "Puli",
    "Pyrenean Shepherds": "Pyrenean Shepherd",
    "Redbone Coonhounds": "Redbone Coonhound",
    "St. Bernard": "Saint Bernard",
    "Scottish Deerhounds": "Scottish Deerhound",
    "Sealyham Terriers": "Sealyham Terrier",
    "Shetland Sheepdog (Sheltie)": "Shetland Sheepdog",
    "Siberian Huskie": "Siberian Husky",
    "Skye Terriers": "Skye Terrier",
    "Soft Coated Wheaten Terrier": "Soft-Coated Wheaten Terrier",
    "Spinone Italiano": "Spinoni Italiani",
    "Sussex Spaniels": "Sussex Spaniel",
    "Swedish Vallhunds": "Swedish Vallhund",
    "Wirehaired Vizslas": "Wirehaired Vizsla"
}

In [15]:
# Now that we have a change dictionary, let's update the breeds in each of the dictionaries so that they match
popularity_df['Breed'] = popularity_df['Breed'].map(correction_dict).fillna(popularity_df['Breed'])
breed_info_df['Breed'] = breed_info_df['Breed'].map(correction_dict).fillna(breed_info_df['Breed'])
best_show_df['Breed'] = best_show_df['Breed'].map(correction_dict).fillna(best_show_df['Breed'])
dog_iq['Breed'] = dog_iq['Breed'].map(correction_dict).fillna(dog_iq['Breed'])

In [16]:
# Now let's try to figure out how many rows are missing data for height and weight
print("Found {} rows without weight data".format(len(best_show_df[best_show_df['weight (lbs)'] == 'no data'])))
print("Found {} rows without height data".format(len(best_show_df[best_show_df['shoulder height (in)'] == 'no data'])))

Found 85 rows without weight data
Found 13 rows without height data


In [17]:
# We are going to try and do some replacements to fill these values by using the data in the breed_info_df
# First thing we have to do is set the index of the dataframes to the breed so we can use that when updating
#best_show_df.set_index('Breed', inplace=True)
best_show_df.head()

,Breed,category,datadog score,POPULARITY IN US,POPULARITY IN US.1,"LIFETIME COST, $",5 LIFETIME COST,1 INTELLIGENCE (TRAINABILITY) ranking,INTELLIGENCE (TRAINABILITY) ranking,2 LONGEVITY,...,"food per lifetime, $","Other regular costs, total per lifetime, $","total per year, $","total, per year, £","toys, presents, treats, per year, £","pet sitters, per year, £","grooming, per year, £","vet fees per year, £","kennels per year, £","one offs, $"
0,Border Collie,herding,3.64,45.0,39.0,"$20,143",48%,1,100%,12.52,...,"3,486","13,095","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
1,Border Terrier,terrier,3.61,80.0,61.0,"$22,638",14%,30,70%,14.00,...,"3,898","14,643","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
2,Brittany,sporting,3.54,30.0,30.0,"$22,589",16%,19,80%,12.92,...,"5,171","13,514","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
3,Cairn Terrier,terrier,3.53,59.0,48.0,"$21,992",22%,35,61%,13.84,...,"3,854","14,476","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0
4,Welsh Springer Spaniel,sporting,3.34,130.0,81.0,"$20,224",47%,31,69%,12.49,...,"3,478","13,064","1,046",784.0,121.0,126.0,244.0,177.0,116.0,200.0


In [18]:
#breed_info_df.set_index('Breed', inplace=True)
#breed_info_df.head(15)


In [19]:
#popularity_df.set_index('Breed', inplace=True)
#popularity_df.head()

In [20]:
best_show_df[['weight (lbs)']] = best_show_df[['weight (lbs)']].replace(['no data','NA (3 classes)'], 0)
#best_show_df[best_show_df['weight (lbs)']==0]

In [21]:
best_show_df[['shoulder height (in)']] = best_show_df[['shoulder height (in)']].replace(['no data','NA (3 classes)'], 0)
#best_show_df[best_show_df['shoulder height (in)']==0]

In [22]:
# Now that we have all the data frames with the same index, let's try a replacement for height and weight
breed_info_name_list = list(breed_info_df['Breed'])
for i in range(len(best_show_df)):
    if (best_show_df['weight (lbs)'][i] == 0) and (best_show_df['Breed'][i] in breed_info_name_list) :
        best_show_df['weight (lbs)'][i] = float(breed_info_df[breed_info_df['Breed']==(best_show_df['Breed'][i])]['Mean Weight'])
    if (best_show_df['shoulder height (in)'][i] == 0) and (best_show_df['Breed'][i] in breed_info_name_list) :
        best_show_df['shoulder height (in)'][i] = float(breed_info_df[breed_info_df['Breed']==(best_show_df['Breed'][i])]['Mean Height']) 
#best_show_df[['weight (lbs)']]
#best_show_df['weight (lbs)'] = best_show_df.apply(lambda row: breed_info_df.loc[row.name]['Mean Weight'] 
#                   if ((row['weight (lbs)'] == 0) and 
#                       row.name in breed_info_name_list) else row['weight (lbs)'], axis=1)
#best_show_df['shoulder height (in)'] = best_show_df.apply(lambda row: breed_info_df.loc[row.name]['Mean Height'] 
#                   if (row['shoulder height (in)'] == 0 and row.name in breed_info_name_list) else 
#                                                          row['shoulder height (in)'], axis=1)

In [23]:
# Now let's check to see how we are doing after the replacement
print("Found {} rows without weight data".format(len(best_show_df[best_show_df['weight (lbs)'] == 0])))
print("Found {} rows without height data".format(len(best_show_df[best_show_df['shoulder height (in)'] == 0])))

Found 17 rows without weight data
Found 1 rows without height data


In [24]:
# The last thing to do would be to merge the popularity_df and best_show_df to create our merged dataframe
merged_df = pd.merge(pd.merge(popularity_df,best_show_df,on='Breed'),dog_iq,on='Breed')
merged_df.head()

,Breed,2018 Rank,2017 Rank,2016 Rank,2015 Rank,2014 Rank,Mean Rank,2014-2015 Change,2015-2016 Change,2016-2017 Change,...,"toys, presents, treats, per year, £","pet sitters, per year, £","grooming, per year, £","vet fees per year, £","kennels per year, £","one offs, $",Classification,obey,reps_lower,reps_upper
0,Labrador Retriever,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,121.0,126.0,244.0,177.0,116.0,200.0,Brightest Dogs,95%,1,4
1,German Shepherd Dog,2,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,...,121.0,126.0,244.0,177.0,116.0,200.0,Brightest Dogs,95%,1,4
2,Golden Retriever,3,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,121.0,126.0,244.0,177.0,116.0,200.0,Brightest Dogs,95%,1,4
3,French Bulldog,4,4.0,6.0,6.0,9.0,5.8,3.0,0.0,2.0,...,121.0,126.0,244.0,177.0,116.0,200.0,Fair Working/Obedience Intelligence,30%,41,80
4,Bulldog,5,5.0,4.0,4.0,4.0,4.4,0.0,0.0,-1.0,...,121.0,126.0,244.0,177.0,116.0,200.0,Lowest Degree of Working/Obedience Intelligence,NaN,81,100


In [25]:
#only contain breed rank,height,weight,iq
sort_df = merged_df[['Breed','Mean Rank','category','weight (lbs)','shoulder height (in)','Classification', 'obey']].sort_values('Mean Rank')
sort_df.replace(np.nan, '>30%', regex=True, inplace=True)
sort_df[:100]

,Breed,Mean Rank,category,weight (lbs),shoulder height (in),Classification,obey
0,Labrador Retriever,1.0,sporting,67.5,23,Brightest Dogs,95%
1,German Shepherd Dog,2.0,herding,82.5,24,Brightest Dogs,95%
2,Golden Retriever,3.0,sporting,60,22.75,Brightest Dogs,95%
4,Bulldog,4.4,non-sporting,45,14,Lowest Degree of Working/Obedience Intelligence,>30%
5,Beagle,5.4,hound,24,14,Lowest Degree of Working/Obedience Intelligence,>30%
3,French Bulldog,5.8,non-sporting,27,11.5,Fair Working/Obedience Intelligence,30%
6,Poodle,7.2,non-sporting,45,16,Brightest Dogs,95%
9,Yorkshire Terrier,8.2,toy,5.5,8,Above Average Working Dogs,70%
7,Rottweiler,8.6,working,100,24.5,Brightest Dogs,95%
10,Boxer,10.0,working,67.5,23.25,Average Working/Obedience Intelligence,50%


In [26]:
# this dataframe is for the top dog breeds in different states

# https://github.com/vega/vega/blob/master/docs/data/population_engineers_hurricanes.csv
states = pd.read_csv("project_data/states.txt")

new_state = pd.DataFrame({'State':['District of Columbia'], 'Top 1':'NaN', 'Top 2':'NaN', 'Top 3':'NaN'}, index=[50])
concat = pd.concat([top_dog_by_state,new_state])
concat = concat.sort_values('State').reset_index(drop=True)
new_state2 = pd.DataFrame({'State':['Puerto Rico'], 'Top 1':'NaN', 'Top 2':'NaN', 'Top 3':'NaN'}, index=[51])
concat = pd.concat([concat,new_state2])
lst=list(states['id'])
concat['id'] = lst
concat = concat.drop([8,51])
concat

,State,Top 1,Top 2,Top 3,id
0,Alabama,Labrador retriever,German shepherd,Beagle,1
1,Alaska,Labrador retriever,German shepherd,Golden retriever,2
2,Arizona,Labrador retriever,German shepherd,Golden retriever,4
3,Arkansas,Labrador retriever,German shepherd,Beagle,5
4,California,Labrador retriever,French bulldog,German shepherd,6
5,Colorado,Labrador retriever,German shepherd,Golden retriever,8
6,Connecticut,Labrador retriever,German shepherd,Golden retriever,9
7,Delaware,Labrador retriever,German shepherd,Golden retriever,10
9,Florida,German shepherd,Labrador retriever,Golden retriever,12
10,Georgia,Labrador retriever,German shepherd,Beagle,13


# Data Visualizaion

## Lifetime cost vs. popularity

The first thing we want to look at is to see the relationship between lifetime cost and popularity. Are the dogs that are the most popular the most expensive? Is there a correlation between popularity and lifetime cost of ownership?

In [27]:
cost_pop_df = merged_df[['Breed', 'LIFETIME COST, $', 'Mean Rank']]
cost_pop_df = cost_pop_df[cost_pop_df['LIFETIME COST, $'] != 'no data']
cost_pop_df['LIFETIME COST, $'] = cost_pop_df['LIFETIME COST, $'].str.replace('$', '')
cost_pop_df['LIFETIME COST, $'] = cost_pop_df['LIFETIME COST, $'].str.replace(',', '').astype(int)
alt.Chart(cost_pop_df).mark_circle(size=50).encode(
    x=alt.X('LIFETIME COST, $', title="Lifetime Cost of Ownership ($)", scale=alt.Scale(domain=[12000, 28000])),
    y=alt.Y('Mean Rank', title='Mean Popularity Rank, 2014-2018'),
    tooltip=['Breed', 'LIFETIME COST, $', 'Mean Rank']
).properties(
    height=600,
    width=600, 
    title='Comparison of Lifetime Cost to Popularity').configure_title(
    fontSize=30).configure_axis(labelFontSize=14, titleFontSize=20)

alt.Chart(...)

There does not look to be any trend in the data. The most popular breed, Laborador Retreiver, is in the middle of the pack for lifetime cost of ownership at 21,299. Interestingly, a chihuahua has a very high cost of ownership of 26,250. Maybe there is some relationship between lifespan and cost of ownership. Let's take a look at that

In [28]:
long_pop_df = merged_df[['Breed', '2 LONGEVITY', 'LIFETIME COST, $', 'size category']]

long_pop_df = long_pop_df[long_pop_df['LIFETIME COST, $'] != 'no data']
long_pop_df['LIFETIME COST, $'] = long_pop_df['LIFETIME COST, $'].str.replace('$', '')
long_pop_df['LIFETIME COST, $'] = long_pop_df['LIFETIME COST, $'].str.replace(',', '').astype(int)

long_pop_df = long_pop_df[~long_pop_df['2 LONGEVITY'].isin(['no data', '1.83 - really?'])]
long_pop_df['2 LONGEVITY'] = long_pop_df['2 LONGEVITY'].astype(float)

alt.Chart(long_pop_df).mark_circle(size=50).encode(
    x=alt.X('2 LONGEVITY', title="Average Lifespan (years)", scale=alt.Scale(domain=[5, 17])),
    y=alt.Y('LIFETIME COST, $', title="Lifetime Cost of Ownership ($)", scale=alt.Scale(domain=[12000, 28000])),
    tooltip=['Breed', '2 LONGEVITY', 'LIFETIME COST, $'],
    color='size category'
).properties(
    height=600,
    width=600, 
    title='Comparison of Lifespan to Lifetime Cost').configure_title(
    fontSize=30).configure_axis(labelFontSize=14, titleFontSize=20)

alt.Chart(...)

We do see a trend here however! We can see that the doges with longer lifespans have a higher cost of ownership. We also see some interesting groupings when we color the data by size category as breeds of the same size category tend to be close together. I think it's worth running a linear regression analysis on this data so let's go ahead and do that now.

In [29]:
from sklearn.linear_model import LinearRegression
x = np.array(long_pop_df['2 LONGEVITY']).reshape(-1, 1)
y = long_pop_df['LIFETIME COST, $']
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

coefficient of determination: 0.5945146463559632
intercept: 6695.224984920511
slope: [1188.09205831]


With a CoD of around 0.6, we can see there is a positive correlation between the average lifespan in years of a dog and the lifetime cost of ownership.

## Trends in popularity data

Now let's take a look at the most popular dog breeds. We are interested in knowing which dogs had the largest change in popularity from year to year.

In [30]:
pop_trend_df = merged_df[['Breed', '2018 Rank', '2017 Rank', '2016 Rank', '2015 Rank', '2014 Rank']]
pop_trend_df.rename(columns={'2018 Rank': '2018', 
                             '2017 Rank': '2017', 
                             '2016 Rank': '2016', 
                             '2015 Rank': '2015', 
                             '2014 Rank': '2014'}, inplace=True)

pop_trend_df = pop_trend_df.melt(id_vars=['Breed'], var_name='Year', value_name='Rank')
mean_df = popularity_df[['Breed', 'Mean Rank']]
pop_trend_df = pop_trend_df.merge(mean_df, on='Breed')

In [31]:
alt.Chart(pop_trend_df[pop_trend_df['Mean Rank'] <= 10]).mark_line(size=3).encode(
    x=alt.X('Year'),
    y=alt.Y('Rank', sort='descending'),
    color='Breed'
).properties(height=600, width=600, title='Popularity Track of Top 10 Most Popular Breeds: 2014-2018').configure_title(
    fontSize=30).configure_axis(labelFontSize=14, titleFontSize=20)

alt.Chart(...)

Here we can see that the top 3 most popular dogs have stayed consistent within the past 3 years: Laborador Retriever, German Shepherd Dog, and Golden Retriever. There have been some changes however as we can see the French Bulldog is increasing in popularity while both the Yorkshire Terrier and Boxer are decreasing in popularity. Let's take a look at the breeds with the biggest year to year changes.

In [32]:
pop_change_df = merged_df[['Breed', '2014-2015 Change', '2015-2016 Change',  '2016-2017 Change', '2017-2018 Change']]
pop_change_df.rename(columns={'2017-2018 Change': '2018', 
                              '2016-2017 Change': '2017', 
                              '2015-2016 Change': '2016', 
                              '2014-2015 Change': '2015'}, inplace=True)

pop_change_df["Mean Change"] = pop_change_df.apply(lambda row: np.mean(row[1:]), axis=1)
pop_change_df["Max Change"] = pop_change_df.apply(lambda row: np.max(np.abs(row[1:])), axis=1)

change_df = pop_change_df[['Breed', 'Mean Change', 'Max Change']]

pop_change_df.drop(columns=['Mean Change', 'Max Change'], inplace=True)
pop_change_df = pop_change_df.melt(id_vars=['Breed'], var_name='Year', value_name='Rank')
pop_change_df = pop_change_df.merge(change_df, on='Breed')

top_change_limit = sorted(set(change_df['Max Change']))[-11:][0]

In [33]:
filtered_change_df = pop_change_df[(pop_change_df['Max Change'] >= top_change_limit) & (pop_change_df['Max Change'] < 100)]
filtered_breed_df = pop_trend_df[pop_trend_df['Breed'].isin(filtered_change_df['Breed'])]

filtered_breed_df.rename(columns={"Year": "Popularity Year", "Rank": "Popularity Rank", "Mean Rank": "Mean Popularity Rank"}, inplace=True)
filtered_change_df.rename(columns={"Year": "Change Year", "Rank": "Popularity Change"}, inplace=True)

complete_pop_change_frame = filtered_breed_df.merge(filtered_change_df, on='Breed')

multi = alt.selection_multi(fields=['Breed'])

base = alt.Chart(complete_pop_change_frame).properties(
    width=600,
    height=600
).add_selection(multi)

change_chart = base.mark_line(size=3).encode(
    x=alt.X('Change Year'),
    y=alt.Y('Popularity Change', title='Change from Previous Year'),
    color=alt.condition(multi,
                        alt.Color('Breed:N'),
                        alt.value('lightgray')),
    tooltip=['Breed', 'Popularity Change']
).properties(title='Popularity Changes for Breeds With Largest Changes: 2014-2018')

change_pop_chart = base.mark_line(size=3).encode(
    x=alt.X('Popularity Year'),
    y=alt.Y('Popularity Rank', sort='descending'),
    color=alt.condition(multi,
                        alt.Color('Breed:N'),
                        alt.value('lightgray')),
    tooltip=['Breed', 'Popularity Rank']
).properties(title='Popularity for Breeds with Largest Changes: 2014-2018')

alt.vconcat(change_chart, change_pop_chart).configure_title(
    fontSize=20).configure_axis(labelFontSize=14, titleFontSize=16)


alt.VConcatChart(...)

## Linear regression to determine predictors of popularity

Using all the different characteristics of the dog breed that we have, we are interested in understanding which factors have the biggest impact of dog popularity.

I don't believe that linear regression is covered under the required courses for Mileston I/II. Because linear regression and other supervised learning techniques were taught during SIADS 524: Supervised Learning which is not a prerequisite for this course, this task is currently out of scope but could be tackled during future courses/analysis.